# 그룹연산
    11-1 데이터 집계
    11-2 데이터 변환
    11-3 데이터 필터링
    11-4 그룹 오프젝트

##### 그룹연산 : 데이터의 '분할(splic) - 반영(Apply) - 결합(Combine)' 과정

### 11-1 데이터 집계 - groupby 메서드

In [2]:
import pandas as pd
df = pd.read_csv('./data/gapminder.tsv', sep='\t')
print(df.head())

       country continent  year  lifeExp       pop   gdpPercap
0  Afghanistan      Asia  1952   28.801   8425333  779.445314
1  Afghanistan      Asia  1957   30.332   9240934  820.853030
2  Afghanistan      Asia  1962   31.997  10267083  853.100710
3  Afghanistan      Asia  1967   34.020  11537966  836.197138
4  Afghanistan      Asia  1972   36.088  13079460  739.981106


In [3]:
#year열 기준 그룹화 > lifeExp 평균
avg_life_exp_by_year = df.groupby('year').lifeExp.mean()    #df.groupby('year')['lifeExp'].mean() 도 같은 결과
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


##### 분할-반영-결합 과정 쪼개보기(feat.groupby메서드)

In [4]:
#groupby메서드에 열이름을 전달하면 '분할'작업이 먼저 일어남
years = df.year.unique()
print(years)

[1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 2002 2007]


In [5]:
#연도별 평균값 구하려면, 각 연도별 데이터 추출이 선과제
    #각 연도별 데이터 추출이 '반영'작업의 한 부분임
y1952 = df.loc[df.year == 1952, :]
print(y1952.head())

        country continent  year  lifeExp       pop    gdpPercap
0   Afghanistan      Asia  1952   28.801   8425333   779.445314
12      Albania    Europe  1952   55.230   1282697  1601.056136
24      Algeria    Africa  1952   43.077   9279525  2449.008185
36       Angola    Africa  1952   30.015   4232095  3520.610273
48    Argentina  Americas  1952   62.485  17876956  5911.315053


In [7]:
#'반영'작업 반복 진행
y1952_mean = y1952.lifeExp.mean()
print(y1952_mean)

y1957 = df.loc[df.year == 1957, :]
y1957_mean = y1957.lifeExp.mean()
print(y1957_mean)
#...
y2007 = df.loc[df.year == 2007, :]
y2007_mean = y2007.lifeExp.mean()
print(y2007_mean)

49.05761971830987
51.507401126760534
67.00742253521126


In [8]:
#'결합'
df2 = pd.DataFrame({'year':[1952, 1957, 2007],
                    '':[y1952_mean, y1957_mean, y2007_mean]})
print(df2)

   year           
0  1952  49.057620
1  1957  51.507401
2  2007  67.007423


##### groupby 메서드와 함께 쓰는 집계 메서드
    메서드              설명
    count               누락값 제외한 데이터 수 리턴
    size                누락값 포함 데이터 수 리턴
    mean                평균값
    std                 표준편차
    min                 최소값
    quantile(q=0.25)    백분위수 25%
    quantile(q=0.50)    백분위수 50% = 중앙값
    quantile(q=0.75)    백분위수 75%
    max                 최대값
    sum                 합
    var                 분산
    sem                 평균의 표준편차
    describe            데이터 수, 평균, 표준편차, 최소, 백분위수(25,50,75%), 최대 값 모두 리턴
    first               첫번째 행 리턴
    last                마지막 행 리턴
    nth                 n번째 행 리턴


##### agg 메서드 : 사용자 함수 & groupby 메서드 조합하기
    라이브러리 메서드로 원하는값을 구할 수 없을때 직접 함수를 정의해 사용

In [9]:
#사용자함수 정의
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    
    return sum / n

In [10]:
#agg메서드를 사용해 사용자 정의 함수와 groupby 메서드를 조합하여 사용
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)   #mean() 메서드 사용과 동일결과 얻음을 확인
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


In [11]:
#인자값 2개 받는 사용자함수
    #첫 인자 열의 평균과 두번째 인차와의 차이
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    mean = sum / n
    return mean - diff_value

In [12]:
#전체 평균 수명
    #두번째 인자로 쓸 값임
global_mean = df.lifeExp.mean()
print(global_mean)

59.47443936619714


In [13]:
#첫 인자로 각 년도의 평균수명, 두번째 인자로 전체 평균수명
    #그 둘의 차이를 구하는 함수(사용자 정의 함수)
        #결과로 평균수명이 증가함을 알 수 있음
agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value = global_mean)
print(agg_mean_diff)

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


##### 여러 집계 메서드 동시 사용 > 집계 메서드를 리스트/딕셔너리에 담아 agg로 전달하면 가능


In [15]:
#agg(함수리스트ver)
import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
print(gdf)

      count_nonzero       mean        std
year                                     
1952            142  49.057620  12.225956
1957            142  51.507401  12.231286
1962            142  53.609249  12.097245
1967            142  55.678290  11.718858
1972            142  57.647386  11.381953
1977            142  59.570157  11.227229
1982            142  61.533197  10.770618
1987            142  63.212613  10.556285
1992            142  64.160338  11.227380
1997            142  65.014676  11.559439
2002            142  65.694923  12.279823
2007            142  67.007423  12.073021


In [18]:
print(df.head())
print()

#agg(함수딕셔너리ver)
gdf_dict = df.groupby('year').agg({'lifeExp':'mean', 'pop':'median', 'gdpPercap':'median'})
print(gdf_dict)

       country continent  year  lifeExp       pop   gdpPercap
0  Afghanistan      Asia  1952   28.801   8425333  779.445314
1  Afghanistan      Asia  1957   30.332   9240934  820.853030
2  Afghanistan      Asia  1962   31.997  10267083  853.100710
3  Afghanistan      Asia  1967   34.020  11537966  836.197138
4  Afghanistan      Asia  1972   36.088  13079460  739.981106

        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334740
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1992  64.160338   8688686.5  4386.085502
1997  65.014676   9735063.5  4781.825478
2002  65.694923  10372918.5  5319.804524
2007  67.007423  10517531.0  6124.371108


### 11-2 데이터 변환
    데이터 변환 메서드는 데이터와 메서드를 일대일로 대응시켜 계산
    따라서 데이터 양이 줄어들지 않음, 말 그대로 데이터를 변환하는데 사용(무슨말이지)

##### 표준점수 계산
    표준점수 : 데이터의 평균과 표준편차의 차이를 의미
    표준점수를 구하면, 변환한 데이터의 평균값은 0, 표준편차는 1이 됨
    이렇게 데이터 표준화가 되어 서로 다른 데이터를 쉽게 비교할 수 있음

In [20]:
#표준점수 계산 함수
def my_zscore(x):
    return (x - x.mean()) / x.std()

In [21]:
#각 연도별 lifeExp열의 표준점수 계산
    #my_zscore 함수 적용 위해 transform 메서드 사용
transform_z = df.groupby('year').lifeExp.transform(my_zscore)
print(transform_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


In [22]:
#my_zscore함수는 데이터를 표준화할뿐, 집계는 하지않음
    #그래서 데이터 양이 줄어들지 않음
print(df.shape)
print(transform_z.shape)
    #데이터 행 개수 변화 없음

(1704, 6)
(1704,)


##### 누락값을 평균값으로 처리하기
    누락값은 상황에 따라 평균으로 처리하는게 좋을때가 있음

In [28]:
import seaborn as sns
import numpy as np

np.random.seed(42)  #랜덤패턴 seed(42)로 고정
tips_10 = sns.load_dataset('tips').sample(10)   #seaborn라이브러리 tips데이터집합 10개의 행 데이터 로드
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN    #total_bill열 임의의 4개 누락값처리
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [29]:
#tips_10 데이터 남성(Male)이 더 많음
    #단순히 total_bill 열의 누락값을 평균값으로 채우면 왜곡될 수 있음
        #성별로 그룹화하여 처리
count_sex = tips_10.groupby('sex').count()
print(count_sex)

        total_bill  tip  smoker  day  time  size
sex                                             
Male             4    7       7    7     7     7
Female           2    3       3    3     3     3


In [30]:
#성별 구분하여 total_bill 열의 평균구하는 함수
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [31]:
#성별을 고려한 평균으로, 누락값(NaN)의 자리에 평균을 채운 열(fill_total_bill) 추가
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
print(tips_10)

     total_bill   tip     sex smoker   day    time  size  fill_total_bill
24        19.82  3.18    Male     No   Sat  Dinner     2          19.8200
6          8.77  2.00    Male     No   Sun  Dinner     2           8.7700
153         NaN  2.00    Male     No   Sun  Dinner     4          17.9525
211         NaN  5.16    Male    Yes   Sat  Dinner     4          17.9525
198         NaN  2.00  Female    Yes  Thur   Lunch     2          13.9300
176         NaN  2.00    Male    Yes   Sun  Dinner     2          17.9525
192       28.44  2.56    Male    Yes  Thur   Lunch     2          28.4400
124       12.48  2.52  Female     No  Thur   Lunch     2          12.4800
9         14.78  3.23    Male     No   Sun  Dinner     2          14.7800
101       15.38  3.00  Female    Yes   Fri  Dinner     2          15.3800


### 11-3 데이터 필터링

##### 데이터 필터링 : filter메서드

In [36]:
tips = sns.load_dataset('tips')
print(tips.head())
print(tips.shape)

   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4
(244, 7)


In [37]:
#size열은 각 테이블 번호의 주문 수
print(tips['size'].value_counts())
    #1,5,6번 테이블 주문이 매우 적다는 것을 알수있음

2    156
3     38
4     37
5      5
1      4
6      4
Name: size, dtype: int64


In [41]:
#30회 미만은 표본수가 너무 적어 제외시키려 함
tips_filtered = tips.\
    groupby('size').\
        filter(lambda x: x['size'].count() >= 30)

In [42]:
#13개의 데이터 제외됨
print(tips_filtered.shape)
print(tips_filtered['size'].value_counts())

(231, 7)
2    156
3     38
4     37
Name: size, dtype: int64


### 11-4 그룹 오브젝트

In [48]:
#groupby 메서드의 결과값 그룹오브젝트 살펴보기
tips_10 = sns.load_dataset('tips').sample(10, random_state=42)
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153       24.55  2.00    Male     No   Sun  Dinner     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
198       13.00  2.00  Female    Yes  Thur   Lunch     2
176       17.89  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [45]:
grouped = tips_10.groupby('sex')
print(grouped)

In [46]:
#groups : 그룹오브젝트에 포함된 그룹 출력
print(grouped.groups)
    #sex열로 그룹화한 데이터 프레임 인덱스 확인 가능
        #해당 그룹오브젝트로 집계, 변환, 필터 등 작업 수행

{'Male': [24, 6, 153, 211, 176, 192, 9], 'Female': [198, 124, 101]}


##### 한번에 그룹 오브젝트 계산하기

In [49]:
#groupd 내에는 평균을 구할수 없는 열(smoker, day, time) 등이 있음
    #이런 그룹 오브젝트에 mean 사용가능 > 파이썬에서 자동적으로 평균계산 가능한 열을 골라서 연산해줌
avgs = grouped.mean()
print(avgs)
    #평균 계산 가능한 열(total_bill, tip, size)만 골라서 자동으로 계산해줌

        total_bill       tip      size
sex                                   
Male         20.02  2.875714  2.571429
Female       13.62  2.506667  2.000000


In [50]:
print(tips_10.columns)

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')


##### 그룹 오브젝트 활용
    그룹오브젝트 데이터 추출하고 반복하기
    
    추가참고자료 : https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html

In [51]:
#get_group : 그룹오브젝트 특정 데이터 추출
    #sex열로 그룹 > 성별 여성데이터만 추출
female = grouped.get_group('Female')
print(female)


     total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [52]:
#그룹오브젝트 반복문 사용
for sex_group in grouped:
    print(sex_group)
    
    #결과를 자세히 보면 튜플값으로 넘어옴

('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)


In [53]:
#sex_group 정보 출력
for sex_group in grouped:
    print('the type is: {}\n'.format(type(sex_group)))
    print('the length is: {}\n'.format(len(sex_group)))
    
    first_element = sex_group[0]
    print('the first element is: {}\n'.format(first_element))
    print('it has a type of: {}\n'.format(type(sex_group[0])))
    
    second_element = sex_group[1]
    print('the second element is: {}\n'.format(second_element))
    print('it has a type of: {}\n'.format(type(second_element)))
    
    print('what we have:')
    print(sex_group)
    
    break

the type is: <class 'tuple'>

the length is: 2

the first element is: Male

it has a type of: <class 'str'>

the second element is:      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2

it has a type of: <class 'pandas.core.frame.DataFrame'>

what we have:
('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  

In [54]:
### 여러 열 사용해 그룹오브젝트 만들고 계산하기
    #여러 열 그룹화 > 리스트에 열 이름 담아 groupby
bill_sex_time = tips_10.groupby(['sex', 'time'])
group_avg = bill_sex_time.mean()

print(group_avg)

               total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000


In [55]:
#group_avg 자료형 : 데이터프레임
#group_avg 포함된 열 : total_bill, tip, size

print(type(group_avg))
print(group_avg.columns)

<class 'pandas.core.frame.DataFrame'>
Index(['total_bill', 'tip', 'size'], dtype='object')


In [56]:
#멀티인덱스(MultiIndex) : 여러 인덱스를 담고 있는 인덱스
print(group_avg.index)

MultiIndex([(  'Male',  'Lunch'),
            (  'Male', 'Dinner'),
            ('Female',  'Lunch'),
            ('Female', 'Dinner')],
           names=['sex', 'time'])


In [57]:
#reset_index : 데이터프레임 인덱스가 MultiIndex인 경우에 데이터프레임 인덱스 새로 부여하는 메서드
group_method = tips_10.groupby(['sex', 'time']).mean().reset_index()
print(group_method)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000


In [58]:
#reset_index 메서드 대신 as_index=False 인자를 통해서도 위와 동일한 결과 얻을 수 있음
group_param = tips_10.groupby(['sex', 'time'], as_index=False).mean()
print(group_param)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000
